In [1]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np

# Load the data

In [2]:
order_list = pd.read_csv('preprocessed_OrderList.csv')
freight_rates = pd.read_csv('preprocessed_FreightRates.csv')
wh_costs = pd.read_csv('preprocessed_WhCosts.csv')
wh_capacities = pd.read_csv('preprocessed_WhCapacities.csv')
products_per_plant = pd.read_csv('preprocessed_ProductsPerPlant.csv')
vmi_customers = pd.read_csv('preprocessed_VmiCustomers.csv')
plant_ports = pd.read_csv('preprocessed_PlantPorts.csv')

# Merge products_per_plant to include product-plant compatibility

In [3]:
merged_data = order_list.merge(products_per_plant, on=['Plant Code', 'Product ID'], how='left')



# Add a flag for customer-plant compatibility

In [4]:
vmi_customers['Customer'] = vmi_customers['Customers'].str.split(',')
vmi_customers = vmi_customers.explode('Customer')
merged_data = merged_data.merge(vmi_customers[['Plant Code', 'Customer']], on=['Plant Code', 'Customer'], how='left', indicator=True)
merged_data['Customer_Compatibility'] = np.where(merged_data['_merge'] == 'both', 1, 0)
merged_data.drop(columns=['_merge'], inplace=True)


# Merge plant_ports to include plant-port connectivity

In [5]:
merged_data = merged_data.merge(plant_ports, on='Plant Code', how='left')

# Merge wh_capacities to include plant daily capacity

In [6]:
merged_data = merged_data.merge(wh_capacities, left_on='Plant Code', right_on='Plant ID', how='left')

# Merge wh_costs to include warehousing cost per unit

In [7]:
merged_data = merged_data.merge(wh_costs, left_on='Plant Code', right_on='WH', how='left')


# Merge freight_rates to include freight cost information

In [8]:
merged_data = merged_data.merge(freight_rates, left_on=['Carrier', 'Origin Port', 'Destination Port'], right_on=['Carrier', 'orig_port_cd', 'dest_port_cd'], how='left')


# Dropping columns that won't be used as features

In [9]:
columns_to_drop = ['Order ID', 'Order Date', 'Plant ID', 'WH', 'orig_port_cd', 'dest_port_cd', 'minimum cost', 'Carrier type']
columns_to_drop = [col for col in columns_to_drop if col in merged_data.columns]
merged_data.drop(columns=columns_to_drop, inplace=True)

# Converting categorical columns to string to ensure uniform data type

In [10]:
categorical_columns = ['Carrier', 'Origin Port', 'Destination Port', 'Service Level', 'Customer', 'Product ID', 'Plant Code']
for col in categorical_columns:
    merged_data[col] = merged_data[col].astype(str)

# Handling missing values (for simplicity, we'll fill with zeroes or use imputation strategies)

In [11]:
merged_data.fillna(0, inplace=True)

# Separating features and target variable

In [12]:
X = merged_data.drop(columns=['Plant Code'])
y = merged_data['Plant Code'].astype(str)

# ColumnTransformer to apply different preprocessors to different columns

In [13]:
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Define a pipeline for feature selection and model training

In [14]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),  # Preprocess the data
    ('selector', SelectKBest(f_classif)),  # Feature selection
    ('classifier', RandomForestClassifier())  # Classifier
])

# Parameter grid for GridSearchCV

In [15]:
param_grid = {
    'selector__k': [5, 10, 15, 'all'],  # Number of features to select
    'classifier__n_estimators': [50, 100, 200],  # Number of trees in forest
    'classifier__max_depth': [None, 10, 20],  # Maximum depth of the tree
}

# GridSearchCV for finding the best model

In [16]:
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')

In [18]:
# Print column names to check for correct column names
print("OrderList columns:", order_list.columns)
print("FreightRates columns:", freight_rates.columns)
print("WhCosts columns:", wh_costs.columns)
print("WhCapacities columns:", wh_capacities.columns)
print("ProductsPerPlant columns:", products_per_plant.columns)
print("VmiCustomers columns:", vmi_customers.columns)
print("PlantPorts columns:", plant_ports.columns)

OrderList columns: Index(['Order ID', 'Order Date', 'Origin Port', 'Carrier', 'TPT',
       'Service Level', 'Ship ahead day count', 'Ship Late Day count',
       'Customer', 'Product ID', 'Plant Code', 'Destination Port',
       'Unit quantity', 'Weight'],
      dtype='object')
FreightRates columns: Index(['Carrier', 'orig_port_cd', 'dest_port_cd', 'minm_wgh_qty',
       'max_wgh_qty', 'svc_cd', 'minimum cost', 'rate', 'mode_dsc',
       'tpt_day_cnt', 'Carrier type'],
      dtype='object')
WhCosts columns: Index(['WH', 'Cost/unit'], dtype='object')
WhCapacities columns: Index(['Plant ID', 'Daily Capacity '], dtype='object')
ProductsPerPlant columns: Index(['Plant Code', 'Product ID'], dtype='object')
VmiCustomers columns: Index(['Plant Code', 'Customers', 'Customer'], dtype='object')
PlantPorts columns: Index(['Plant Code', 'Port'], dtype='object')
